In [2]:
import pandas as pd
import os
import glob
from IPython.display import display

# --- CONFIGURATION ---
TARGET_CITIES = ["London", "Paris", "Beijing", "Tokyo", "Berlin"]

# Path logic: We are in 'src/', so we go up one level to reach project root, then into 'data/'
PROJECT_ROOT = os.path.dirname(os.getcwd())
RAW_DATA_DIR = os.path.join(PROJECT_ROOT, "data", "raw")
OUTPUT_FILE = os.path.join(PROJECT_ROOT, "data", "processed", "filtered_data.parquet")

# 1. FIND ALL CSV FILES
csv_files = glob.glob(os.path.join(RAW_DATA_DIR, "*.csv"))
all_dataframes = []

if not csv_files:
    print(f"Error: No CSV files found in {RAW_DATA_DIR}")
else:
    print(f"Found {len(csv_files)} CSV files. Starting processing...")
    
    for file_path in csv_files:
        filename = os.path.basename(file_path)
        # The WAQI datasets often have 4 lines of comments at the top.
        # We use skiprows=4 to start reading at the header line.
        df = pd.read_csv(file_path, skiprows=4)
            
        # Clean column names (remove hidden spaces)
        df.columns = df.columns.str.strip()
            
        # Validate required columns exist after skipping headers
        required = ['Date', 'City', 'Specie']
        if all(col in df.columns for col in required):
            # Normalize city names
            df['City'] = df['City'].astype(str).str.title()
            filtered_chunk = df[df['City'].isin(TARGET_CITIES)].copy()
                
            if not filtered_chunk.empty:
                all_dataframes.append(filtered_chunk)
                print(f"  -> Added {len(filtered_chunk)} rows from {filename}")
            else:
                print(f"  -> Skipping {filename}: Could not find columns {required}")


    # 2. CONSOLIDATE AND SAVE
    if all_dataframes:
        print("\nConsolidating and saving...")
        final_df = pd.concat(all_dataframes, ignore_index=True)
        
        # Ensure Date is in datetime format
        final_df['Date'] = pd.to_datetime(final_df['Date'])
        
        # Sort chronologically
        final_df = final_df.sort_values(by=['City', 'Date', 'Specie'])

        # Create output directory
        os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)
        
        # Save as Parquet
        final_df.to_parquet(OUTPUT_FILE, index=False)
        
        print(f"Success! Saved {len(final_df)} rows to {OUTPUT_FILE}")
        print(f"Cities in dataset: {final_df['City'].unique()}")
        display(final_df.head())
    else:
        print("\nNo matching data found for target cities. Check if the city names in CSV match your list.")

Found 25 CSV files. Starting processing...
  -> Added 5812 rows from waqi-covid19-airqualitydata-2015H1.csv
  -> Added 5883 rows from waqi-covid19-airqualitydata-2016H1.csv
  -> Added 5527 rows from waqi-covid19-airqualitydata-2017H1.csv
  -> Added 7566 rows from waqi-covid19-airqualitydata-2018H1.csv
  -> Added 6097 rows from waqi-covid19-airqualitydata-2019Q1.csv
  -> Added 6239 rows from waqi-covid19-airqualitydata-2019Q2.csv
  -> Added 6617 rows from waqi-covid19-airqualitydata-2019Q3.csv
  -> Added 6590 rows from waqi-covid19-airqualitydata-2019Q4.csv
  -> Added 6117 rows from waqi-covid19-airqualitydata-2020Q1.csv
  -> Added 6803 rows from waqi-covid19-airqualitydata-2020Q2.csv
  -> Added 6684 rows from waqi-covid19-airqualitydata-2020Q3.csv
  -> Added 6730 rows from waqi-covid19-airqualitydata-2020Q4.csv
  -> Added 6701 rows from waqi-covid19-airqualitydata-2021Q1.csv
  -> Added 6726 rows from waqi-covid19-airqualitydata-2021Q2.csv
  -> Added 6735 rows from waqi-covid19-airquali

,Date,Country,City,Specie,count,min,max,median,variance
3992,2014-12-29,CN,Beijing,co,429,2.8,90.4,32.6,4266.95
3753,2014-12-29,CN,Beijing,no2,432,2.4,114.2,51.0,3901.05
4357,2014-12-29,CN,Beijing,o3,354,0.9,39.5,3.7,932.37
4298,2014-12-29,CN,Beijing,pm10,376,22.0,735.0,118.0,63652.90
3685,2014-12-29,CN,Beijing,pm25,421,18.0,564.0,190.0,133562.00
